In [ ]:
import csv
import gc
import cv2
from tensorflow import keras
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
import neptune
from tensorflow.python.client import device_lib
from PIL import Image
import tensorflow as tf
import os
from keras.layers import LeakyReLU
from pathlib import Path
from decouple import config
import matplotlib
from matplotlib import cm
import itertools
from pathlib import Path
from scipy.io import savemat

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

In [ ]:
########################################################################################################################
# Load dataset
########################################################################################################################
case_num = 390
def load_dataset():    
    
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical_test_set/')            
    Full_W_dataset = np.load('GT_Num_output_case_%d.npy' % case_num)
    print(Full_W_dataset.shape)
    CS_arr = np.load('Sample_Num_output_case_%d.npy' % case_num)
    print(CS_arr.shape)
    x_test = CS_arr
    y_test = Full_W_dataset    
    return x_test, y_test  

In [ ]:
########################################################################################################################
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Saeed_model/h5_model')
new_dim = 32
CR = (new_dim ** 2) / (69 ** 2)  # Compression Ratio
rescale_factor = int(512 / new_dim)
model_name = 'Sub_pix_latest.h5' 
model = load_model(model_name, compile=False)
model.summary()


In [ ]:
def testing():   
    x_test, y_test = load_dataset()                 
    print(x_test.shape)
    print(y_test.shape)
    
    prediction = model.predict(x_test, batch_size=1)
    prediction = np.asarray(prediction)
    frames = x_test.shape[0]
    
    print(frames)
    for i in range(frames):
        SR_pred = prediction[i].astype('float32')            
        original = x_test[i].astype('float32')
        GT_label_input = y_test[i].astype('float32')
        ############################################################################################################
        plt.figure(figsize=(1, 1), dpi=512)
        plt.gca().set_axis_off()
        plt.axis('off')
        plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
        plt.margins(0, 0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        ############################################################################################################          
        SR_pred = SR_pred * 255.0
        # SR_pred= SR_pred.astype(np.float32)
        plt.imshow(SR_pred)        
        plt.axis('off')

        path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Saeed_model/num/%d_output/' % case_num
        Path(path).mkdir(parents=True, exist_ok=True)
        os.chdir(path)      
        #cv2.imwrite('SR_Pred_output_%d_frame_output_%d.png' % (i+1, case_num), SR_pred)
        plt.savefig('Saeed_SR_Pred_frame_%d_output_%d.png' % (i+1, case_num))
        plt.close('all')

In [ ]:
if __name__ == '__main__':
    testing()

In [ ]:
# os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical_results_Full_wavefield/381_output/')
# for i in range(1, 513):
#     data = cv2.imread('SR_Pred_output_%d_frame_output_381.png' % i, 0)
#     print(np.max(data))